In [1]:
import sklearn
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import collections
from itertools import chain
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from sklearn.compose import make_column_selector, make_column_transformer
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

# 1. Data
### 1.1 Load/Import Data

In [4]:
trades = pd.read_csv(Path("../data/raw_data/AAPL_trades.csv"))
quotes = pd.read_csv(Path("../data/raw_data/AAPL_quotes.csv"))

In [5]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283504 entries, 0 to 283503
Data columns (total 17 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Unnamed: 0                              283504 non-null  int64  
 1   Time                                    283504 non-null  object 
 2   Date                                    283504 non-null  object 
 3   Exchange                                283504 non-null  object 
 4   Symbol                                  283504 non-null  object 
 5   Trade_Volume                            283504 non-null  int64  
 6   Trade_Price                             283504 non-null  float64
 7   Sale_Condition                          283504 non-null  object 
 8   Source_of_Trade                         283504 non-null  object 
 9   Trade_Stop_Stock_Indicator              0 non-null       float64
 10  Trade_Correction_Indicator              2835

Note: All column information of trades and quotes data and valid entries for each column can be found at https://www.nyse.com/publicdocs/nyse/data/Daily_TAQ_Client_Spec_v3.0.pdf

In [6]:
trades.head()

,Unnamed: 0,Time,Date,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator
0,0,2020-01-02 04:00:00.064010,2020-01-02,P,AAPL,3801,295.05,@ T,N,NaN,0,1185,1,,40000063617792,NaN,1
1,1,2020-01-02 04:00:02.828485,2020-01-02,P,AAPL,1,295.08,@FTI,N,NaN,0,1195,2,,40002828108800,NaN,1
2,2,2020-01-02 04:00:06.250392,2020-01-02,Q,AAPL,6,295.25,@ TI,N,NaN,0,1197,1,,40006250366823,NaN,0
3,3,2020-01-02 04:00:06.429757,2020-01-02,P,AAPL,1,295.08,@ TI,N,NaN,0,1198,3,,40006429377792,NaN,0
4,4,2020-01-02 04:00:28.894835,2020-01-02,P,AAPL,3,295.10,@ TI,N,NaN,0,1205,4,,40028894459136,NaN,0


In [7]:
quotes.head()

,Unnamed: 0,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,National_BBO_Indicator,FINRA_BBO_Indicator,FINRA_ADF_MPID_Indicator,Quote_Cancel_Correction,Source_Of_Quote,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Participant_Timestamp,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator,Date,YearMonth
0,0,2020-01-02 04:00:00.065165,P,AAPL,278.00,7.0,0.00,0.0,R,2228,2,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,40000064785664,NaN,NaN,,2020-01-02,202001
1,1,2020-01-02 04:00:00.065167,P,AAPL,278.00,14.0,0.00,0.0,R,2229,2,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,40000064787456,NaN,NaN,,2020-01-02,202001
2,2,2020-01-02 04:00:00.065170,P,AAPL,293.72,9.0,0.00,0.0,R,2230,2,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,40000064790784,NaN,NaN,,2020-01-02,202001
3,3,2020-01-02 04:00:00.065681,P,AAPL,293.72,9.0,327.56,1.0,R,2231,4,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,40000065302272,NaN,NaN,,2020-01-02,202001
4,4,2020-01-02 04:00:00.065738,P,AAPL,293.72,9.0,320.00,1.0,R,2232,4,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,40000065358592,NaN,NaN,,2020-01-02,202001


### 1.2 Data Visualization & Preliminary Analysis

### 1.3 Data Cleaning

Before we move towards feature generation and building machine learning models, we have to clean the dataset. The necessary steps to clean the trades and quotes data include:
1. Get rid of unnecessary columns.
2. Get rid of the invalid trades and quotes.
3. Event reconstruction.
4. Only keep the natural best bid/offer or last MQU.
5. Assign Last Active Quote and assign to the corresponding trade.

## SCIKIT-LEARN DESIGN

https://arxiv.org/pdf/1309.0238.pdf

Scikit-Learn’s API is remarkably well designed. These are the main design components of Scikit-Learn.

All objects share a consistent and simple interface:

### Estimators

Any object that can estimate some parameters based on a dataset is called an estimator (e.g., a SimpleImputer is an estimator). The estimation itself is performed by the fit() method, and it takes a dataset as a parameter, or two for supervised learning algorithms—the second dataset contains the labels. Any other parameter needed to guide the estimation process is considered a hyperparameter (such as a SimpleImputer’s strategy), and it must be set as an instance variable (generally via a constructor parameter).

### Transformers

Some estimators (such as a SimpleImputer) can also transform a dataset; these are called transformers. Once again, the API is simple: the transformation is performed by the transform() method with the dataset to transform as a parameter. It returns the transformed dataset. This transformation generally relies on the learned parameters, as is the case for a SimpleImputer. All transformers also have a convenience method called fit_transform(), which is equivalent to calling fit() and then transform() (but sometimes fit_transform() is optimized and runs much faster).


### Predictors

Finally, some estimators, given a dataset, are capable of making predictions; they are called predictors. For example, the LinearRegression model in the previous chapter was a predictor: given a country’s GDP per capita, it predicted life satisfaction. A predictor has a predict() method that takes a dataset of new instances and returns a dataset of corresponding predictions. It also has a score() method that measures the quality of the predictions, given a test set (and the corresponding labels, in the case of supervised learning algorithms).

### ...

Reference to the base classes for all estimators in scikit-learn can be found at: https://github.com/scikit-learn/scikit-learn/blob/9aaed4987/sklearn/base.py#L153

In [8]:
from generators import *
from hf_methods import *

In [9]:
clean_pipeline = make_pipeline(
    CleanData()
)

In [10]:
clean_trades = clean_pipeline.fit_transform(trades)
clean_trades.head()

test


,Exchange,Symbol,Trade_Volume,Trade_Price,Sale_Condition,Source_of_Trade,Trade_Stop_Stock_Indicator,Trade_Correction_Indicator,Sequence_Number,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Trade_Reporting_Facility_TRF_Timestamp,Trade_Through_Exempt_Indicator
2020-01-02 09:15:03.454000,D,AAPL,4,296.25,@ TI,N,NaN,0,16067,706,N,2020-01-02 09:15:03.454000,9.150346e+13,0
2020-01-02 09:15:06.873000,D,AAPL,1,296.25,@ TI,N,NaN,0,16087,707,N,2020-01-02 09:15:06.873000,9.150688e+13,0
2020-01-02 09:15:07.893000,D,AAPL,100,296.15,@ T,N,NaN,0,16088,708,N,2020-01-02 09:15:07.893000,9.150790e+13,0
2020-01-02 09:15:08.263000,D,AAPL,7,296.25,@ TI,N,NaN,0,16090,709,N,2020-01-02 09:15:08.263000,9.150827e+13,0
2020-01-02 09:15:08.271318,P,AAPL,1,296.15,@ TI,N,NaN,0,16091,1544,,2020-01-02 09:15:08.271318,NaN,0


In [11]:
clean_quotes = clean_pipeline.fit_transform(quotes)
clean_quotes.head()

test


,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,National_BBO_Indicator,FINRA_BBO_Indicator,FINRA_ADF_MPID_Indicator,Quote_Cancel_Correction,Source_Of_Quote,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Participant_Timestamp,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator
2020-01-02 09:15:00.086755,X,AAPL,283.18,1.0,309.95,1.0,R,202469,2,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,2020-01-02 09:15:00.086755,NaN,NaN,
2020-01-02 09:15:08.271548,K,AAPL,296.06,1.0,296.33,1.0,R,202696,0,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,2020-01-02 09:15:08.271548,NaN,NaN,
2020-01-02 09:15:08.271574,Z,AAPL,295.37,3.0,296.82,2.0,R,202697,0,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,2020-01-02 09:15:08.271574,NaN,NaN,
2020-01-02 09:15:08.301537,K,AAPL,296.06,1.0,296.33,1.0,R,202699,0,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,2020-01-02 09:15:08.301537,NaN,NaN,
2020-01-02 09:15:08.301539,Z,AAPL,295.37,3.0,296.82,2.0,R,202698,0,NaN,NaN,NaN,N,NaN,0,NaN,NaN,NaN,2020-01-02 09:15:08.301539,NaN,NaN,


### 1.4 Reconstructing Events

In [12]:
clean_trades['Is_Quote'] = False
clean_quotes['Is_Quote'] = True
trade_features = ['Symbol', 'Trade_Volume', 'Trade_Price', 'Trade_Id', 'Trade_Reporting_Facility', 'Participant_Timestamp', 'Is_Quote']
quote_features = ['Symbol', 'Bid_Price', 'Bid_Size', 'Offer_Price', 'Offer_Size', 'Participant_Timestamp', 'Is_Quote']

In [13]:
all_events = clean_trades[trade_features].append(clean_quotes[quote_features], ignore_index=True)
all_events.index.name = "RID"
all_events = all_events.sort_values(by=['Participant_Timestamp', all_events.index.name])
all_events.head(10)

,Symbol,Trade_Volume,Trade_Price,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Is_Quote,Bid_Price,Bid_Size,Offer_Price,Offer_Size
RID,,,,,,,,,,,
244031,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:00.086755,True,283.18,1.0,309.95,1.0
0,AAPL,4.0,296.25,706.0,N,2020-01-02 09:15:03.454000,False,NaN,NaN,NaN,NaN
1,AAPL,1.0,296.25,707.0,N,2020-01-02 09:15:06.873000,False,NaN,NaN,NaN,NaN
2,AAPL,100.0,296.15,708.0,N,2020-01-02 09:15:07.893000,False,NaN,NaN,NaN,NaN
3,AAPL,7.0,296.25,709.0,N,2020-01-02 09:15:08.263000,False,NaN,NaN,NaN,NaN
4,AAPL,1.0,296.15,1544.0,,2020-01-02 09:15:08.271318,False,NaN,NaN,NaN,NaN
244032,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:08.271548,True,296.06,1.0,296.33,1.0
244033,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:08.271574,True,295.37,3.0,296.82,2.0
5,AAPL,1.0,296.15,3.0,,2020-01-02 09:15:08.274097,False,NaN,NaN,NaN,NaN


### 1.5 Preprocess Data

All preprossing steps are implemented according to the papaer: The Participant Timestamp: Get The Most Out Of TAQ Data https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3984827 

Trade direction is assigned using the tick test, which can be refered from the paper: Inferring Trade Direction from Intraday Data by Charles M. C. Lee, Mark J. Ready https://www.jstor.org/stable/2328845

#### *For labeling valid quotes using AND(&) operator:

|Is_Quote|valid_quotes|Desired|
|---|---|---|
|False|False|False|
|False|True|False|
|True|False|False|
|True|True|True|

In [14]:
preprocess_pipeline = make_pipeline(
    PreprocessData()
)

In [15]:
df_prepared = preprocess_pipeline.fit_transform(all_events)
df_prepared.head(20)

,Symbol,Trade_Volume,Trade_Price,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Is_Quote,Bid_Price,Bid_Size,Offer_Price,Offer_Size,MOX,Valid_Quotes,Trade_Sign,Participant_Timestamp_f
RID,,,,,,,,,,,,,,,
244031,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:00.086755,True,283.18,1.0,309.95,1.0,0,True,NaN,1.577957e+09
0,AAPL,4.0,296.25,706.0,N,2020-01-02 09:15:03.454000,False,NaN,NaN,NaN,NaN,1,False,1.0,1.577957e+09
1,AAPL,1.0,296.25,707.0,N,2020-01-02 09:15:06.873000,False,NaN,NaN,NaN,NaN,2,False,1.0,1.577957e+09
2,AAPL,100.0,296.15,708.0,N,2020-01-02 09:15:07.893000,False,NaN,NaN,NaN,NaN,3,False,-1.0,1.577957e+09
3,AAPL,7.0,296.25,709.0,N,2020-01-02 09:15:08.263000,False,NaN,NaN,NaN,NaN,4,False,1.0,1.577957e+09
4,AAPL,1.0,296.15,1544.0,,2020-01-02 09:15:08.271318,False,NaN,NaN,NaN,NaN,5,False,-1.0,1.577957e+09
244032,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:08.271548,True,296.06,1.0,296.33,1.0,6,True,NaN,1.577957e+09
244033,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:08.271574,True,295.37,3.0,296.82,2.0,7,True,NaN,1.577957e+09
5,AAPL,1.0,296.15,3.0,,2020-01-02 09:15:08.274097,False,NaN,NaN,NaN,NaN,8,False,-1.0,1.577957e+09


In [16]:
df_prepared[df_prepared['MOX'] == 11]

,Symbol,Trade_Volume,Trade_Price,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Is_Quote,Bid_Price,Bid_Size,Offer_Price,Offer_Size,MOX,Valid_Quotes,Trade_Sign,Participant_Timestamp_f
RID,,,,,,,,,,,,,,,
6,AAPL,50.0,296.15,1439.0,,2020-01-02 09:15:09.340550,False,NaN,NaN,NaN,NaN,11,False,-1.0,1.577957e+09
7,AAPL,70.0,296.12,1440.0,,2020-01-02 09:15:09.340550,False,NaN,NaN,NaN,NaN,11,False,-1.0,1.577957e+09
8,AAPL,100.0,296.12,1441.0,,2020-01-02 09:15:09.340550,False,NaN,NaN,NaN,NaN,11,False,-1.0,1.577957e+09
9,AAPL,71.0,296.11,1442.0,,2020-01-02 09:15:09.340550,False,NaN,NaN,NaN,NaN,11,False,-1.0,1.577957e+09
244036,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:09.340550,True,296.1,28.0,296.25,1.0,11,False,NaN,1.577957e+09
244037,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:09.340550,True,296.1,27.0,296.25,1.0,11,False,NaN,1.577957e+09
244038,AAPL,NaN,NaN,NaN,NaN,2020-01-02 09:15:09.340550,True,296.1,27.0,296.11,1.0,11,True,NaN,1.577957e+09


In [17]:
# import the set_config module from sklearn
from sklearn import set_config

# set the display option for sklearn to 'diagram'
set_config(display='diagram')

# display the pipeline 
preprocess_pipeline

Pipeline(steps=[('preprocessdata', PreprocessData())])

## 2. Feature Generating Pipeline

## 2.1 Generating Features

#### 2.1.1

#### 2.1.2 Return and Imbalance

#### 2.1.3

## 2.2 Pipeline

### 2.2.2 Feature Generation Pipeline

In [18]:
df_test = df_prepared.copy()[:40000]
df_test['Trade_Volume'] = df_test['Trade_Volume'].apply(lambda t: t if not np.isnan(t) else 0)
df_test.reset_index(drop=True, inplace=True)
df_test.head()

,Symbol,Trade_Volume,Trade_Price,Trade_Id,Trade_Reporting_Facility,Participant_Timestamp,Is_Quote,Bid_Price,Bid_Size,Offer_Price,Offer_Size,MOX,Valid_Quotes,Trade_Sign,Participant_Timestamp_f
0,AAPL,0.0,NaN,NaN,NaN,2020-01-02 09:15:00.086755,True,283.18,1.0,309.95,1.0,0,True,NaN,1.577957e+09
1,AAPL,4.0,296.25,706.0,N,2020-01-02 09:15:03.454000,False,NaN,NaN,NaN,NaN,1,False,1.0,1.577957e+09
2,AAPL,1.0,296.25,707.0,N,2020-01-02 09:15:06.873000,False,NaN,NaN,NaN,NaN,2,False,1.0,1.577957e+09
3,AAPL,100.0,296.15,708.0,N,2020-01-02 09:15:07.893000,False,NaN,NaN,NaN,NaN,3,False,-1.0,1.577957e+09
4,AAPL,7.0,296.25,709.0,N,2020-01-02 09:15:08.263000,False,NaN,NaN,NaN,NaN,4,False,1.0,1.577957e+09


In [19]:
RETURN_CALENDAR_SPAN = 5
RETURN_TRANSACTION_SPAN = None
RETURN_VOLUME_SPAN = None

CALENDAR_DELTAS = [(0,.1),(.1,.2),(.2,.4),(.4,.8),(.8,1.6),(1.6,3.2),(3.2,6.4),(6.4,12.8),(12.8,25.6)]
TRANSACTION_DELTAS = [(0,1),(1,2),(2,4),(4,8),(8,16),(16,32),(32,64),(64,128),(128,256)]
VOLUME_DELTAS = [(0,100),(100,200),(200,400),(400,800),(800,1600),(1600,3200),(3200,6400),(6400,12800),(12800,25600)]

In [20]:
# df_test = parent_generator_ret_imb(df_test, CALENDAR_DELTAS[:1])

In [21]:
params = {
    'return_span': RETURN_CALENDAR_SPAN,
    'clock_mode': 'calendar',
    'deltas': CALENDAR_DELTAS
}

In [23]:
feature_pipeline = make_pipeline(
    FeatureGeneration()
)
df = feature_pipeline.fit_transform(df_test, params)